<a href="https://colab.research.google.com/github/dominique-nshimyimana/Karat-GC/blob/master/lstm_car_state_action.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Colab: TensorFlow and GoogleDrive

In [0]:
%tensorflow_version 2.x

In [68]:
from google.colab import drive
mounted = '/drive'
drive.mount(mounted)

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


# NN state Evaluation

In [0]:
import tensorflow as tf
import numpy as np
from os import listdir
from os.path import isfile, join
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, LSTM, SimpleRNN, GRU, Dense
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import RepeatVector
from tensorflow.keras.layers import TimeDistributed


def createCarStateActionLSTM((look_back, XS_shape[-1]), y_shape[-1]):
    model = Sequential()
    model.add(LSTM(units = 100, return_sequences = True, activation='relu',input_shape=(look_back, XS_shape[-1])))
    model.add(LSTM(units = 200, return_sequences = True, activation='relu'))
    model.add(LSTM(units = 24, dropout = 0.2, activation='relu', return_sequences=True))
    model.add(LSTM(units = 21, dropout = 0.2, activation='relu', return_sequences=True))
    model.add(TimeDistributed(Dense(units = y_shape[-1], activation='linear')))
    return model


class CarStateActionLSTM(tf.keras.Model):
  '''
    # Add a LSTM layer with 128 internal units.
    model.add(LSTM(128))
    # The output of GRU will be a 3D tensor of shape (batch_size, timesteps, units)
    model.add(GRU(256, return_sequences=True))
    # The output of SimpleRNN will be a 2D tensor of shape (batch_size, units)
    model.add(SimpleRNN(128))
  '''
  def __init__(self, input_shape, output_shape, dropout = False):

    super(CarStateActionLSTM, self).__init__()
    self.in_shape = input_shape
    self.out_shape = output_shape
    self.dropout = dropout
    
    self.model = Sequential()
    self.model.add(LSTM(units = np.prod(self.in_shape), return_sequences = True, activation='relu'))
    self.model.add(LSTM(units = self.out_shape*np.prod(self.input_shape), return_sequences = True, activation='relu'))
    self.model.add(LSTM(units = self.out_shape*4, dropout = 0.2, activation='relu', return_sequences = True))
    self.model.add(LSTM(units = self.out_shape*2, dropout = 0.2, activation='relu', return_sequences = True))
    self.model.add(TimeDistributed(Dense(units = self.output_shape, activation='linear')))


  def call(self, x):
    x = tf.nn.dropout(inputs, rate=self.dropout)
    return self.model(x)

# Prepare Data

### Data Functions

In [0]:
# Load dat from folder
# CSV form: Dataset
# Timestamp, x, y, yaw, xvel, yvel, omega, accel, brake, steering
def load_states_actions(data_dir, cutoff_beginning = 300, cutoff_end = 1000, norm=True):
  data_files = [join(data_dir, f) for f in listdir(data_dir) if isfile(join(data_dir, f)) and ".directory" not in f]

  xs_states = []
  xs_actions = []
  ys = []

  for f in data_files:
      print(f)
      data = np.loadtxt(f, delimiter=', ', skiprows=1, dtype=np.float32)[cutoff_beginning:-cutoff_end, :]
      x_states = [data[i, 1:-3] for i in range(len(data))]
      x_actions = [data[i, -3:] for i in range(len(data))]
      y = x_states[1:]
      x_states, x_actions = x_states[:-1], x_actions[:-1]
      xs_states += x_states
      xs_actions += x_actions

  xs_states = np.vstack(xs_states)
  xs_actions = np.vstack(xs_actions)

  scaler = MinMaxScaler()
  xs_states = scaler.fit_transform(xs_states)
  xs_actions = scaler.fit_transform(xs_actions)
  return xs_states, xs_actions

# convert an array of values into a dataset matrix
# State(s) at a given time t (or time series until t) and Y is the state at the next time (t + 1).
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		dataX.append(np.squeeze(dataset[i:(i+look_back)])) #dataset[i:(i+look_back), 0]
		dataY.append(dataset[i + look_back]) #dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)

### Load and Preapre

In [71]:
# Load Data
data_dir = "/drive/My Drive/neuronyte_logging"
xs_states, xs_actions = load_states_actions(data_dir=data_dir)

/drive/My Drive/neuronyte_logging/NeuroNyte_1585748620.316643
/drive/My Drive/neuronyte_logging/NeuroNyte_1585745385.311904


In [72]:
# Split data
x_states_train, x_states_test, x_actions_train, x_actions_test = train_test_split(xs_states, xs_actions, test_size=0.10)

# How many steps/state in back from t and # How many steps/state in future/forward from t
look_back = 10
look_forward = 1

# reshape into X=t and Y=t+1 for states
x_states_train, y_train = create_dataset(x_states_train, look_back)
x_states_test, y_test = create_dataset(x_states_test, look_back)

# Size
print(x_states_train.shape, x_actions_train.shape, y_train.shape)
print(x_states_test.shape, x_actions_test.shape, y_test.shape)

# Visualize X=t and Y=t+1
t = 3
print(y_test[t])
print(x_states_test[t+1])

XS_shape, XA_shape, y_shape = x_states_train.shape, x_actions_train.shape, y_train.shape
X = np.array(x_states_train).reshape(XS_shape[0], look_back, XS_shape[-1])
Y = np.array(y_train).reshape(y_shape[0], look_forward, y_shape[-1])

(948071, 10, 6) (948082, 3) (948071, 6)
(105332, 10, 6) (105343, 3) (105332, 6)
[0.37424535 0.13924694 0.7270725  0.7409239  0.5350734  0.49997193]
[[0.05207235 0.23790044 0.94030714 0.60957265 0.15868369 0.5006329 ]
 [0.75319105 0.18277627 0.13920838 0.45517683 0.46346644 0.4999416 ]
 [0.6460107  0.05012929 0.24931204 0.00763676 0.49099526 0.49997658]
 [0.9517953  0.39264026 0.02845061 0.41546392 0.04266897 0.49989057]
 [0.8439419  0.11120158 0.7106651  0.9785719  0.6029468  0.5003241 ]
 [0.4543416  0.28463978 0.23745495 0.3281563  0.4864741  0.49997187]
 [0.20290941 0.19910985 0.55314004 0.57617784 0.7438341  0.49931416]
 [0.674147   0.5118116  0.935324   0.56995326 0.23333722 0.5005042 ]
 [0.07806629 0.4379715  0.48367113 0.4736774  0.83786714 0.50003844]
 [0.37424535 0.13924694 0.7270725  0.7409239  0.5350734  0.49997193]]


# Initialize Model

In [101]:
decay=tf.keras.optimizers.schedules.ExponentialDecay(0.0004, 20000, 0.99)
optimizer = tf.keras.optimizers.Adam(learning_rate=decay)

print((look_back, XS_shape[-1]))
#model = CarStateActionLSTM((look_back, XS_shape[-1]), y_shape[-1])
model = createCarStateActionLSTM((look_back, XS_shape[-1]), y_shape[-1])

(10, 6)


# Training

In [0]:
print(x_states_train.shape, x_actions_train.shape, y_train.shape)
EPOCHS = 100
model.compile(optimizer = optimizer, loss='mse')

# checkpoint
'''
filepath=join(mounted, "My Drive/NeuralModel/lstm_car_state/weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5")
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
# Fit the model
history = model.fit(X, Y, epochs=EPOCHS, validation_split=0.2, verbose=1, batch_size=20000, callbacks=callbacks_list)
'''

history = model.fit(X, Y, epochs=EPOCHS, validation_split=0.2, verbose=1, batch_size=2000)
model.save(join(mounted, 'My Drive/NeuralModel/lstm_car_state/lstm_fin.h5'))

(948071, 10, 6) (948082, 3) (948071, 6)
Epoch 1/100
380/380 [==============================] - 36s 95ms/step - loss: 0.0876 - val_loss: 0.0512
Epoch 2/100
380/380 [==============================] - 35s 93ms/step - loss: 0.0512 - val_loss: 0.0506
Epoch 3/100
380/380 [==============================] - 36s 94ms/step - loss: 0.0508 - val_loss: 0.0506
Epoch 4/100
380/380 [==============================] - 36s 94ms/step - loss: 0.0507 - val_loss: 0.0506
Epoch 5/100
380/380 [==============================] - 35s 93ms/step - loss: 0.0506 - val_loss: 0.0505
Epoch 6/100
380/380 [==============================] - 35s 93ms/step - loss: 0.0506 - val_loss: 0.0506
Epoch 7/100
380/380 [==============================] - 36s 94ms/step - loss: 0.0506 - val_loss: 0.0505
Epoch 8/100
380/380 [==============================] - 36s 93ms/step - loss: 0.0506 - val_loss: 0.0506
Epoch 9/100
380/380 [==============================] - 36s 94ms/step - loss: 0.0505 - val_loss: 0.0505
Epoch 10/100
380/380 [===========

# Testing

In [0]:
print(x_states_test.shape, x_actions_test.shape, y_test.shape)
test_input = x_states_train[11121:11126]
test_input = test_input.reshape((test_input.shape[0], 1, test_input.shape[1]))

test_output = model.predict(test_input, verbose=0)
print(test_output.shape, 'OUT Shape')
print(test_input.shape, 'IN Shape')

In [0]:
for zx in zip(test_input[:, :, :2],test_output[:, :, :2]):
  print(zx)

# Visualization of result

In [0]:
# x, y, ... from shape=(6,)
xs_states_inpt = np.squeeze(test_input[:, :, :2])
xs_states_pred = np.squeeze(test_output[:, :, :2])

plt.plot(*zip(*xs_states_inpt), 'r')
plt.plot(*zip(*xs_states_pred),'b')
plt.xlabel('x pos')
plt.ylabel('y pos')
plt.show()

In [0]:
# Nice to look at
!pip install gym
import gym
env = gym.make('CartPole-v0')
env.reset()
for _ in range(1000):
    #env.render()
    env.step(env.action_space.sample()) # take a random action
env.close()
help(env.__doc__)

## Docs


1.   https://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/
2.   https://stackabuse.com/solving-sequence-problems-with-lstm-in-keras/
3.   https://towardsdatascience.com/reinforcement-learning-w-keras-openai-dqns-1eed3a5338c
4.   https://keras.io/guides/functional_api/,  https://keras.io/api/models/model/ and https://keras.io/api/layers/recurrent_layers/

